In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report


train_set_df = pd.read_csv("Train_set.csv")


X_train = train_set_df.drop(columns=["Severity"])
y_train = train_set_df["Severity"]


dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)


test_data = pd.read_csv('Test_Set.csv')

# Separate features and target variable for the test set
X_test = test_data.drop(columns=["Severity"])
y_test = test_data["Severity"]

y_pred = dt_model.predict(X_test)

dt_accuracy = accuracy_score(y_test, y_pred)
dt_recall = recall_score(y_test, y_pred, average='macro')
dt_f1 = f1_score(y_test, y_pred, average='macro')

print(f"Decision Tree Accuracy: {dt_accuracy:.4f}")
print(f"Decision Tree Recall: {dt_recall:.4f}")
print(f"Decision Tree F1 Score: {dt_f1:.4f}")


print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Decision Tree Accuracy: 0.6356
Decision Tree Recall: 0.6002
Decision Tree F1 Score: 0.4036


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score


train_set_df = pd.read_csv("Train_set.csv")

X_train = train_set_df.drop(columns=["Severity"])
y_train = train_set_df["Severity"]


train_set_df = pd.read_csv("Test_set.csv")

X_test = train_set_df.drop(columns=["Severity"])
y_test = train_set_df["Severity"]

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)



y_pred = rf_model.predict(X_test)


rf_accuracy = accuracy_score(y_test, y_pred)
rf_recall = recall_score(y_test, y_pred, average='macro')
rf_f1 = f1_score(y_test, y_pred, average='macro')

print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print(f"Random Forest Recall: {rf_recall:.4f}")
print(f"Random Forest F1 Score: {rf_f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()


train_set_df = pd.read_csv("Train_set.csv")


X_train = train_set_df.drop(columns=["Severity"])
y_train = label_encoder.fit_transform(train_set_df["Severity"])


test_data = pd.read_csv('Test_Set.csv')

# Separate features and target variable for the test set
X_test = test_data.drop(columns=["Severity"])
y_test = label_encoder.fit_transform(train_set_df["Severity"])


size: int = min(len(X_test), len(X_train))

X_train, y_train, X_test, y_test = X_train[:size], y_train[:size], X_test[:size], y_test[:size]


xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)


y_pred = xgb_model.predict(X_test)


xgb_accuracy = accuracy_score(y_test, y_pred)
xgb_recall = recall_score(y_test, y_pred, average='macro')
xgb_f1 = f1_score(y_test, y_pred, average='macro')

print(f"XGBoost Accuracy: {xgb_accuracy:.4f}")
print(f"XGBoost Recall: {xgb_recall:.4f}")
print(f"XGBoost F1 Score: {xgb_f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

XGBoost Accuracy: 0.4740
XGBoost Recall: 0.3327
XGBoost F1 Score: 0.3224


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, top_k_accuracy_score, average_precision_score
from sklearn.metrics import accuracy_score, recall_score, f1_score


train_set_df = pd.read_csv("Train_set.csv")

X_train = train_set_df.drop(columns=["Severity"])
y_train = train_set_df["Severity"]


train_set_df = pd.read_csv("Test_set.csv")

X_test = train_set_df.drop(columns=["Severity"])
y_test = train_set_df["Severity"]


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), activation='relu', max_iter=1000, random_state=42)

mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

print("\nClassification Report:")
print(classification_report(y_test, y_pred))